外部資料

In [30]:
import pandas as pd
import numpy as np
from glob import glob, iglob

df = pd.DataFrame()

# iglob使用較少memory
for csv in iglob('../0821/*_top500_clean.csv'):
    tmp_df = pd.read_csv(csv)
    df = df.append(tmp_df, ignore_index=True, sort=False)

In [3]:
len(df)

4769

內部資料(飯店經緯度)

In [31]:
df_gps = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data_gps.csv")

總共1500多家飯店

In [40]:
df_gps.shape

(1565, 5)

內部資料(飯店名稱)

In [32]:
df_name = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data.csv")
df_name = df_name[['hotel_id', 'name']].copy()

In [35]:
len(df_name.name.value_counts()) # 只有900家飯店知道名稱

889

只保留不重複的部分

In [33]:
df_name = df_name.drop_duplicates()

### 結合內部資料

In [34]:
df_gps = pd.merge(df_name, df_gps, left_on=['hotel_id'], right_on=['hotel_id'], how='outer')

緯度為null

In [7]:
df_gps.longitude.isna().sum()

232

In [3]:
df_gps.latitude.isna().sum()

73

經度為0

In [9]:
len(df_gps.loc[df_gps.latitude == 0])

364

In [11]:
len(df_gps.loc[df_gps.longitude == 0])

200

### 移除經緯度為0或是null的飯店

In [35]:
df_gps_clean = df_gps.dropna(subset=['latitude', 'longitude'])

In [36]:
# 僅刪除經緯度
df_gps_clean = df_gps_clean[df_gps_clean.latitude!=0]
df_gps_clean = df_gps_clean[df_gps_clean.longitude!=0]

In [51]:
df_gps_clean.isna().sum()

hotel_id       0
name         372
country      196
city         339
latitude       0
longitude      0
dtype: int64

In [43]:
df_gps_clean.country.value_counts()

台灣          541
tw          182
TW          123
TAIWAN       42
MALAYSIA     14
Taiwan        7
Malaysia      7
日本            6
jp            3
台灣省           2
台湾            1
GREECE        1
my            1
中華民國          1
台灣TAIWAN      1
Name: country, dtype: int64

In [42]:
df_gps_clean.city.value_counts()

台南市        135
宜蘭縣         74
屏東縣         58
南投縣         45
花蓮          43
          ... 
瑞穗           1
Okinawa      1
Kythira      1
彰化           1
白沙鄉          1
Name: city, Length: 98, dtype: int64

---
### 處理新的google map gps

In [37]:
# 留有google map gps的
df.dropna(inplace=True)
len(df)

2038

In [38]:
def get_first(x, length):
    return x[:length]

df['gmap_lat'] = df['gmap_lat'].astype('str').apply(get_first, args=(7,)).astype('float')
df['gmap_lng'] = df['gmap_lng'].astype('str').apply(get_first, args=(8,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(8,)).astype('float')

In [39]:
df.duplicated(subset=['gmap_lat', 'gmap_lng'], keep=False).sum()

271

In [40]:
df_tmp = pd.merge(df, df_gps_clean, left_on=['gmap_lat', 'gmap_lng'], right_on=['latitude', 'longitude'])
df_tmp.drop_duplicates(subset=['gmap_lat', 'gmap_lng'])

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,gmap_lat,gmap_lng,hotel_id,name,country,city,latitude,longitude
0,伊笛幔休閒民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.973978,121.605408,花蓮市富裕十一街79號,3.5,花蓮,花蓮市,"NT$1,330 - NT$2,722",5.0,30.0,23.9972,121.5981,1321,花蓮鳶尾花民宿,台灣,花蓮市,23.9972,121.5981
1,看見七星潭,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.490971,121.325439,七星街106號,3.0,花蓮,新城,"NT$1,918 - NT$4,670",5.0,4.0,24.0266,121.6306,995,NaN,台灣,HUALIEN,24.0266,121.6306
2,海想念,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.930998,121.575706,大廈街3號,3.5,花蓮,花蓮市,"NT$1,268 - NT$3,526",4.5,3.0,23.9746,121.6154,1346,海想念民宿,台灣,NaN,23.9746,121.6154
3,山水商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.040842,121.621796,花蓮市國聯三路73號,2.5,花蓮,花蓮市,"NT$1,052 - NT$1,547",4.0,3.0,23.9904,121.6048,1297,NaN,台灣,NaN,23.9904,121.6048
4,享海民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.971722,121.597687,中正路169巷1-7號,3.5,花蓮,花蓮市,-NT$1,5.0,14.0,23.9717,121.5976,962,享海民宿,台灣,花蓮,23.9717,121.5976
5,海芸民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.975121,121.614861,花蓮市北濱街43號,3.5,花蓮,花蓮市,"NT$1,763 - NT$3,155",4.5,5.0,23.9751,121.6148,1091,NaN,台灣,NaN,23.9751,121.6148
6,幸福加油站民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.976233,121.602608,信義街93號,3.0,花蓮,花蓮市,"NT$1,206 - NT$3,340",5.0,4.0,23.9762,121.6026,256,幸福加油站民宿,TW,花蓮市,23.9762,121.6026
8,城市家園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.980110,121.605888,花蓮市國威里大同街30-2號,3.0,花蓮,花蓮市,"NT$1,949 - NT$4,361",4.5,9.0,23.9801,121.6058,189,NaN,台灣,花蓮,23.9801,121.6058
9,碧夏民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.974060,121.600357,自由街110巷7號,3.5,花蓮,花蓮市,"NT$2,134 - NT$3,990",5.0,6.0,23.9770,121.6062,884,碧夏民宿,台灣,花蓮市,23.9770,121.6062
10,法蘿拉民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.025528,121.604889,花蓮市中強街151號,3.0,花蓮,花蓮市,"NT$1,268 - NT$3,464",5.0,2.0,23.9692,121.6019,1087,法蘿拉民宿,台灣,NaN,23.9692,121.6019


---
tripadvisor預設經緯度

1. 使用內部資料全部的小數點

In [5]:
def get_first(x, length):
    return x[:length]

# 把經緯度成跟內部資料位數一樣
df['lat'] = df['lat'].astype('str').apply(get_first, args=(8,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(10,)).astype('float')

2. 內外部資料都只用小數點4位看能不能和

In [75]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(6,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(6,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(7,)).astype('float')

3. 小數點第四位後做四捨五入 **會完全和不了**

In [55]:
def get_round(x, length):
    return round(x, length)

df['lat'] = df['lat'].apply(get_round, args=(5,))
df['lng'] = df['lng'].apply(get_round, args=(6,))
df_gps_clean['latitude'] = df_gps_clean['latitude'].apply(get_round, args=(5,))
df_gps_clean['longitude'] = df_gps_clean['longitude'].apply(get_round, args=(6,))

找有沒有經緯度完全重複的

In [56]:
df.duplicated(subset=['lat', 'lng'], keep=False).sum()

1659

In [57]:
df.duplicated(subset=['lat', 'lng']).sum()

1060

In [47]:
df_gps_clean.duplicated(subset=['latitude', 'longitude'], keep=False).sum()

39

經緯度同時重疊

In [58]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].sort_values(by=['lat']).sort_values(by=['lng'])

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
764,麗馨精品商旅七賢館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,前金區市中一路229號,3.0,高雄,前金,"NT$817 - NT$2,169",4.0,18.0
574,奇異果快捷旅店-九如店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,三民區九如一路790號,3.0,高雄,三民,"NT$849 - NT$2,012",4.0,155.0
535,巴黎商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,新興區自立二路8號,3.0,高雄,新興,"NT$1,257 - NT$2,075",3.5,31.0
643,芳橙汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,小港區中安路616號,3.0,高雄,小港,"NT$1,069 - NT$1,509",3.0,1.0
745,六合轉角737館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.622,120.283,南台路六合夜市 - 美麗島捷運站旁,2.0,高雄,新興,"NT$849 - NT$1,886",4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...
1398,金色年代旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.121,121.861,板橋區重慶路66號12樓,3.0,新北,板橋,"NT$1,257 - NT$2,798",4.0,10.0
3661,大里驛青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,頭城鎮濱海路6段317-2號,2.0,宜蘭,頭城,"NT$1,980 - NT$3,018",5.0,16.0
3943,我家商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,羅東鎮中正路186之2號 3樓,2.5,宜蘭,羅東,"NT$1,100 - NT$3,018",0.0,0.0
1461,九份聽山,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.015,121.944,瑞芳區基山街219之4號,3.0,新北,瑞芳,"NT$943 - NT$2,703",3.0,11.0


In [19]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].groupby(['lat', 'lng']).agg('count')

title  uri  hotel_address  hotel_star  hotel_city  \
lat    lng                                                          
21.912 120.847      2    2              2           2           2   
       120.848      2    2              2           2           2   
21.918 120.844      2    2              2           2           2   
21.920 120.844      2    2              2           2           2   
21.925 120.832      2    2              2           2           2   
...               ...  ...            ...         ...         ...   
25.168 121.445      3    3              3           3           3   
25.169 121.445      2    2              2           2           2   
25.171 121.447      4    4              4           4           4   
25.180 121.689      2    2              2           2           2   
25.255 121.511      2    2              2           2           2   

                hotel_section  price_range  avg_rating  comment_count  
lat    lng                                                             
21.912 120.847              2            2           2              2  
       120.848              2            2           2              2  
21.918 120.844              2            2           2              2  
21.920 120.844              2            2           2              2  
21.925 120.832              2            2           2              2  
...                       ...          ...         ...            ...  
25.168 121.445              3            3           3              3  
25.169 121.445              2            2           2              2  
25.171 121.447              4            4           4              4  
25.180 121.689              2            2           2              2  
25.255 121.511              2            2           2              2  

[599 rows x 9 columns]

經度重疊

In [32]:
df[df['lat'].isin(df['lat'][df['lat'].duplicated()])]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
3,嘉義優遊商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.463,西區中山路617號11樓,3.0,嘉義,東區,"NT$1,163 - NT$2,734",3.5,41.0
39,楓雅SPA 汽車旅館 (大雅館),https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.431,大雅路二段256號,3.5,嘉義,東區,"NT$1,194 - NT$1,697",4.0,1.0
45,香緹汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,西區八德路2號,3.0,嘉義,西區,"NT$1,980 - NT$2,922",3.5,4.0
70,波士頓大飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,東區中正路673號,3.0,嘉義,西區,"NT$691 - NT$1,508",3.0,11.0
72,雙鳳大旅社,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.99303,121.593,復興街5號,2.0,花蓮,花蓮市,"NT$597 - NT$2,766",3.5,11.0
...,...,...,...,...,...,...,...,...,...,...,...
4028,田野小徑,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.80044,121.756,礁溪鄉份尾一路3號,3.5,宜蘭,礁溪,"NT$2,075 - NT$3,395",4.0,1.0
4029,玫瑰莊園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67588,121.751,冬山鄉梅花路742號,3.5,宜蘭,冬山,"NT$1,980 - NT$4,527",0.0,0.0
4030,夜市橙堡渡假旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.82802,121.774,新民路19號,3.5,宜蘭,宜蘭市,"NT$1,320 - NT$2,515",4.0,12.0
4033,賓成精品商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67275,121.809,西後街26-1號,3.0,宜蘭,宜蘭市,"NT$1,163 - NT$2,389",3.0,9.0


### 嘗試能不能concat

內部資料的單一經緯度卻對應到多間外部資料的飯店

In [9]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).head()

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,country,city,latitude,longitude
0,花蓮北吉光輕旅青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,花蓮市中山路601巷1弄19號,2.0,花蓮,花蓮市,"NT$817 - NT$1,949",5.0,432.0,1482,台灣,花蓮市,23.988,121.599
1,希臘仙境美宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,花蓮市北濱街102-2號,3.0,花蓮,花蓮市,"NT$1,603 - NT$3,018",5.0,1.0,1482,台灣,花蓮市,23.988,121.599
2,美麗晨曦民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,吉安鄉太昌村明義二街16號,3.0,花蓮,吉安,"NT$2,483 - NT$4,244",5.0,235.0,1482,台灣,花蓮市,23.988,121.599
3,海洋飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.604,花蓮市國聯三路58號,3.0,花蓮,花蓮市,"NT$1,509 - NT$2,043",4.0,1.0,1297,台灣,NaN,23.990,121.604
4,太魯閣蘇西小空間,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.604,新城鄉新城村中正路20號,2.5,花蓮,新城,"NT$1,069 - NT$2,672",3.5,14.0,1297,台灣,NaN,23.990,121.604


能和的飯店數目

In [64]:
len(pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']))

759

檢查是否有同一間飯店被重複

In [24]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).title.value_counts()

花蓮樂活亞綠亞民宿    4
12號民宿        4
鐵道旅人民宿       4
愛格斯民宿        4
微笑拉菲草        4
            ..
墾丁一號旅店       1
隆德旅客之家       1
嘉蓮小棧         1
小羊房民宿        1
台北國際飯店       1
Name: title, Length: 568, dtype: int64

內部資料本來也有GPS重複的問題

In [76]:
df_tmp = pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude'])
df_tmp[df_tmp['title'].isin(df_tmp['title'][df_tmp['title'].duplicated()])]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,name,country,city,latitude,longitude
11,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.605,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,795,船長不在家民宿,TW,NaN,23.990,121.605
12,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.605,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,796,NaN,台灣,NaN,23.990,121.605
13,踩風民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,吉安鄉福興村11鄰福興七街500號,3.0,花蓮,吉安,"NT$2,515 - NT$3,961",5.0,24.0,1283,小松鼠民宿,台灣,花蓮縣,23.988,121.602
14,踩風民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,吉安鄉福興村11鄰福興七街500號,3.0,花蓮,吉安,"NT$2,515 - NT$3,961",5.0,24.0,1078,NaN,台灣,NaN,23.988,121.602
15,七星燈塔民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,新城鄉七星街59號,3.0,花蓮,花蓮市,"NT$2,389 - NT$3,835",4.5,9.0,1283,小松鼠民宿,台灣,花蓮縣,23.988,121.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,海灘風情,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,頭城鎮外澳里濱海路二段68號,2.5,宜蘭,頭城,"NT$2,483 - NT$3,835",3.0,3.0,1172,石頭屋蝴蝶農場,台灣,宜蘭縣,24.650,121.736
730,香草熊河濱會館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,五結鄉二結路357巷39弄16號,3.5,宜蘭,五結,"NT$1,163 - NT$2,892",0.0,0.0,690,香草森林,台灣,NaN,24.650,121.736
731,香草熊河濱會館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,五結鄉二結路357巷39弄16號,3.5,宜蘭,五結,"NT$1,163 - NT$2,892",0.0,0.0,1172,石頭屋蝴蝶農場,台灣,宜蘭縣,24.650,121.736
742,神雕雅築庭園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.677,121.797,五結鄉福和二路118巷62號,3.5,宜蘭,五結,"NT$2,106 - NT$3,866",5.0,23.0,280,澳玩客民宿,台灣,宜蘭縣,24.677,121.797


實際上和成功的飯店數 (單一經緯度先只保留一家，先不管內外部資料都有GPS重複的問題)

In [81]:
df_tmp.drop_duplicates(subset=['lat', 'lng'])

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,name,country,city,latitude,longitude
0,花蓮北吉光輕旅青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,花蓮市中山路601巷1弄19號,2.0,花蓮,花蓮市,"NT$817 - NT$1,949",5.0,432.0,1482,伊島民宿,台灣,花蓮市,23.988,121.599
3,海洋飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.604,花蓮市國聯三路58號,3.0,花蓮,花蓮市,"NT$1,509 - NT$2,043",4.0,1.0,1297,NaN,台灣,NaN,23.990,121.604
7,君典家民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.600,花蓮市國聯一路215號,3.0,花蓮,花蓮市,"NT$1,540 - NT$3,049",4.5,7.0,1088,君典家民宿,台灣,NaN,23.988,121.600
8,崖上民宿,https://www.tripadvisor.com.tw/Hotel_Review-g2...,23.982,121.592,豐濱鄉新社村新社211號,3.5,花蓮,花蓮的客棧和民宿,-NT$1,4.5,5.0,1024,NaN,台灣,NaN,23.982,121.592
11,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.605,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,795,船長不在家民宿,TW,NaN,23.990,121.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,灣曲時尚渡假會館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.688,121.754,富堵三路101號,3.5,宜蘭,冬山,"NT$4,495 - NT$6,570",5.0,1.0,1356,宜蘭Bobo旅店,tw,NaN,24.688,121.754
755,星宿渡假,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.659,121.780,三星鄉天福村東興路13之5號,3.0,宜蘭,三星,"NT$2,012 - NT$20,150",0.0,0.0,264,NaN,台灣,宜蘭縣,24.659,121.780
756,明水露渡假民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.676,121.745,三星鄉行健三路二段325號,3.5,宜蘭,三星,"NT$2,452 - NT$4,589",5.0,105.0,550,NaN,台灣,宜蘭縣,24.676,121.745
757,忙裡偷閒渡假民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.671,121.803,員山鄉深溝村深洲路200巷9號,3.5,宜蘭,員山,"NT$1,760 - NT$5,375",0.0,0.0,537,NaN,台灣,宜蘭縣,24.671,121.803
